# Inclusive UE

In [1]:
from array import array
import itertools
import time
import math

import sys
#####For NERSC:####
#sys.path.insert(0,'/usr/common/software/rootpy/')
import root_numpy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

#import astroML.plotting as aml
#import iminuit
import pandas as pd
import root_pandas as rpd
from root_pandas import read_root
import ROOT
from ROOT import TH1D
from ROOT import SetOwnership
from matplotlib.ticker import NullFormatter
from ROOT import TLatex
%matplotlib inline                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

Welcome to ROOTaaS 6.06/06


In [9]:
Shower = "NN"
#Shower = "LO"
CorrectedP = True     #FALSE FOR HARDPROBES

if (Shower == "NN"):
    #pPb_File = 'InputData/pPb_SE_NN_Correlation_GMB_Ratio.root'
    #pp_File = 'InputData/pp_SE_NN_Correlation_GMB_Ratio.root'
    pPb_File = 'InputData/pPb_MB_4GeV_Skim_Correlation_NN.root'
    pPb_File = 'InputData/pp_MB_4GeV_Skim_Correlation_NN.root'
    pp_File = 'InputData/pp_SE_NN_Correlation_GMB_Ratio_Inclusive.root'
    pp_File = 'InputData/13d_0GeVTrack_paired_large_0GeVTracks_Correlation_Lambda_0_to_300.root'
    #pp_File = 'InputData/pp_SE_NN_Correlation_GMB_Ratio.root'
    if (CorrectedP):
        purity = 0.2615
    else:
        purity = 0.352546
if (Shower == "LO"):
    pPb_File = 'InputData/13def_SE_L0_Correlation_GMB_Ratio.root'
    pp_File = 'InputData/pPb_SE_L0_Correlation_GMB_Ratio_Track.root'
    #pp_File = "InputData/17q_MB_0GeV_Skim_Correlation_L0.root"
    if (CorrectedP):
        purity = 0.221083
    else:
        purity = 0.271083
        
print purity

#pPb_File = 'InputData/13d_MB_0GeV_NN_15_20.root'
#pp_File = 'InputData/17q_MB_0Gev_NN_15_20.root'
        
#pPb_File = 'InputData/13def_EMax_SE_GMB_Ratio.root'
#pp_File = 'InputData/17q_SE_EMax_Correlation_GMB_Ratio.root'

MC_File = 'InputData/18b10a_pthat_1_2_SE_NN_Correlation_GMB_Ratio.root'
print pp_File
print pPb_File

0.2615
InputData/13d_0GeVTrack_paired_large_0GeVTracks_Correlation_Lambda_0_to_300.root
InputData/pp_MB_4GeV_Skim_Correlation_NN.root


In [3]:
#delta_phi_centers= np.array([0.19634954084936207, 0.5890486225480862, 0.9817477042468103, 1.3744467859455345, 1.7671458676442586, 2.1598449493429825, 2.552544031041707, 2.945243112740431])

####Ommiting first dphi bin######
delta_phi_centers= np.array([0.5890486225480862, 0.9817477042468103, 1.3744467859455345, 1.7671458676442586, 2.1598449493429825, 2.552544031041707, 2.945243112740431])
phi_width = [0.39269908169872414/2]*len(delta_phi_centers)

###Array used for UE Error bar####
ue_error_bar = np.array([0,0.05,0.1,0.15,0.2,0.25,0.3,0.39269908169872414,2*0.39269908169872414])

zTbins = [0.05, 0.07670637, 0.11767734, 0.18053204, 0.27695915, 0.42489062, 0.65183634, 1]
pTbins = [12,15,19,26,40]

#####. Total Number of zT bins, max 7  #####
N_pT_Bins = 4
NzT = 6
N_Eta_Bins = 14
#####  Skip First N zT bins  #####
zT_offset = 0

zT_centers = np.zeros(NzT)
zT_widths = np.zeros(NzT)
for ztbin in range(zT_offset,NzT+zT_offset):
    zT_centers[ztbin-zT_offset] = (zTbins[ztbin]+ zTbins[ztbin+1])/2
    zT_widths[ztbin-zT_offset] = (zTbins[ztbin+1]-zTbins[ztbin])/2

Corrections = [1,1.007,0.982,0.970,0.942,0.830,0.640]
oneminFake = [1,0.982,0.980,0.978,0.970,0.915,0.812]

In [4]:
def Get_NTriggers(filename,ipt, Signal_DNN=True): 
    file = ROOT.TFile(filename)
    if (Signal_DNN == "Inclusive"):
        ntrig_histo = file.Get('N_Inclusive_Triggers_pT%1.0f_%1.0f' %(pTbins[ipt],pTbins[ipt+1]))
    elif (Signal_DNN == "Isolated"):
        ntrig_histo = file.Get('N_Triggers_pT%1.0f_%1.0f' %(pTbins[ipt],pTbins[ipt+1]))
        
    else:
        DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN))
        ntrig_histo = file.Get('N_DNN%i_Triggers_pT%1.0f_%1.0f' %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1]))
    NTriggers = 1
    if not(ntrig_histo == None):
        NTriggers = ntrig_histo.GetEntries()
    file.Close()
    return NTriggers

In [5]:
def GetEtaProj(filename,prfx,ipt, izt, Signal_DNN=True):
    
    file = ROOT.TFile(filename)

    N_Eta_Bins = 14
    
    if (Signal_DNN == "Inclusive"):
        histo2D = file.Get('Inclusive_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                        %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],100*zTbins[izt+1]))
        
        Eta_Axis = histo2D.GetYaxis()
        EtaProjection = histo2D.ProjectionY('Inclusive_EtaProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                      %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],
                                        100*zTbins[izt+1]),0,-1)
    
    else:
        DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN)) #convert bool to DNN_1 (Sgn) or DNN_2 (Bkgd)
        histo2D = file.Get('DNN%i_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                            %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],100*zTbins[izt+1]))
        
        Eta_Axis = histo2D.GetYaxis()
        EtaProjection = histo2D.ProjectionY('DNN%i_EtaProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                      %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],
                                        100*zTbins[izt+1]),0,-1)
    
    EtaProjection.SetDirectory(0)
    #EtaProjection.Scale(1.0/1.2)
    
    #per trigger yield
    ntriggers = Get_NTriggers(filename,ipt, Signal_DNN)
    if not(ntriggers == None):
        EtaProjection.Scale(1.0/ntriggers)
    
    file.Close()
    
    return EtaProjection

In [6]:
def GetEta_array(filename,prfx,ipt,izt,Signal_DNN=True):
    
    file = ROOT.TFile(filename)
    
    if (Signal_DNN == "Inclusive"):
        histo2D = file.Get('Inclusive_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                        %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],100*zTbins[izt+1]))
        
        Eta_Axis = histo2D.GetYaxis()
        EtaProjection = histo2D.ProjectionY('Inclusive_EtaProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                      %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],
                                        100*zTbins[izt+1]),0,-1)\
        
    elif (Signal_DNN == "Isolated"):
        histo2D = file.Get('Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                        %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],100*zTbins[izt+1]))
        
        Eta_Axis = histo2D.GetYaxis()
        EtaProjection = histo2D.ProjectionY('EtaProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                      %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],
                                        100*zTbins[izt+1]),0,-1)


    
    ntriggers = Get_NTriggers(filename,ipt, Signal_DNN)
    if not(ntriggers == None):
        EtaProjection.Scale(1.0/ntriggers)
    
    eta_array = root_numpy.hist2array(EtaProjection)
    return eta_array

In [7]:
Inclusive_eta_array = np.zeros((N_pT_Bins,NzT,N_Eta_Bins))
Isolated_eta_array = np.zeros((N_pT_Bins,NzT,N_Eta_Bins))

for ptbin in range(4):
    #if(ptbin > 0): continue
    #ptbin = ptbin+2
    print("        ptbin %i:"%(ptbin))
    print("")
    print("Isolated       Inclusive")

    for ztbin in range(6):
    
        #grab
        Inclusive_eta_array[ptbin][ztbin] = GetEta_array(pp_File,"pp",ptbin,ztbin,"Inclusive") #no subtraction, change DNN
        Isolated_eta_array[ptbin][ztbin] = GetEta_array(pp_File,"pp",ptbin,ztbin,"Isolated")

        Inclusive_Diff = Inclusive_eta_array[ptbin][ztbin][:3].sum()-Inclusive_eta_array[ptbin][ztbin][11:].sum()
        Isolated_Diff = Isolated_eta_array[ptbin][ztbin][:3].sum()-Isolated_eta_array[ptbin][ztbin][11:].sum()
        print("%f       %f"%(Inclusive_Diff,Isolated_Diff))
    print("")
#print(sig_eta_array)
print(Inclusive_eta_array[1][0][:3].sum()-Inclusive_eta_array[1][0][11:].sum())


        ptbin 0:

Isolated       Inclusive
0.011544       0.017079
0.008828       0.013673
0.006256       0.008369
0.001853       0.003063
0.000317       -0.000595
0.000665       0.000596

        ptbin 1:

Isolated       Inclusive
0.007063       0.007949
0.003977       0.002699
0.002863       0.001602
0.000976       0.001151
0.001177       0.000881
0.000525       0.000347

        ptbin 2:

Isolated       Inclusive
0.002609       0.004687
0.001652       0.001849
0.000693       0.001563
-0.000166       0.000003
0.000187       0.000171
0.000247       -0.000151

        ptbin 3:

Isolated       Inclusive
0.000161       0.000319
-0.000190       0.000465
0.000055       -0.000334
0.000423       0.000476
0.000364       0.000110
0.000076       -0.000076

0.00706334996263


## Eta Projections

In [10]:
for ptbin in range(4):
    #if(ptbin > 0): continue
    #ptbin = ptbin+2
    canvases = ROOT.TCanvas("canv_phi","canv_phi",3500,2500)
    canvases.Divide(4,3)
    
    for ztbin in range(6):
    
        #grab
        hsig = GetEtaProj(pp_File,"pp",ptbin,ztbin,"Inclusive") #no subtraction, change DNN
        hbkg = GetEtaProj(pp_File,"pp",ptbin,ztbin,"Inclusive")
 
        #same scale
        ymax = 1.2*max(hsig.GetMaximum(),hbkg.GetMaximum())
        hsig.GetYaxis().SetRangeUser(0,ymax)
        hbkg.GetYaxis().SetRangeUser(0,ymax)    
    
        #Signal
        canvases.cd(ztbin*2+1)
        hsig.SetMarkerStyle(15)
        #hsig.SetMarkerSize(4)
        hsig.SetMarkerColor(4)
        hsig.Draw()
        
        #Background
        canvases.cd(ztbin*2+2)
        hbkg.SetMarkerStyle(15)
        #hbkg.SetMarkerSize(4)
        hbkg.SetMarkerColor(2)
        hbkg.Draw()
        #legend.AddEntry(hbkg,"Low DNN","l")
        #legend.Draw("same")
    
    canvases.Draw()   
    canvases.SaveAs("pics/Eta_Same_Mix_Projections_Trigger_pt_%i_Inclusive.png"%(ptbin))

TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: canv_phi


AttributeError: 'TObject' object has no attribute 'GetYaxis'

Error in <TFile::TFile>: file InputData/13d_0GeVTrack_paired_large_0GeVTracks_Correlation_Lambda_0_to_300.root does not exist


# Numbers of Pairs in zT bins

In [ ]:
Shower = "NN"

pPb_File = 'InputData/pPb_SE_NN_Correlation_GMB_Ratio_UnWeight.root'
pp_File = 'InputData/pp_SE_NN_Correlation_GMB_Ratio_UnWeight.root'   
Files = [pPb_File,pp_File]
Systems = ["pPb","pp"]
purity = [0.276899,0.358741,0.456807,0.476192]

NpT = 4
pTbins = [12,15,19,26,40]
zTbins = [0.05, 0.07670637, 0.11767734, 0.18053204, 0.27695915, 0.42489062, 0.65183634, 1]
#####. Total Number of zT bins, max 7  #####
NzT = 6
#####  Skip First N zT bins  #####
zT_offset = 0


zT_centers = np.zeros(NzT)
zT_widths = np.zeros(NzT)
for ztbin in range(zT_offset,NzT+zT_offset):
    zT_centers[ztbin-zT_offset] = (zTbins[ztbin]+ zTbins[ztbin+1])/2
    zT_widths[ztbin-zT_offset] = (zTbins[ztbin+1]-zTbins[ztbin])
    

pT_centers = np.zeros(NpT)
pT_widths = np.zeros(NpT)
for ptbin in range(NpT):
    pT_centers[ptbin] = (pTbins[ptbin]+ pTbins[ptbin+1])/2
    pT_widths[ptbin] = (pTbins[ptbin]-pTbins[ptbin+1])/1.25
print(pT_centers)

In [ ]:
def Get_NTriggers(filename,ipt, Signal_DNN=True): 
    file = ROOT.TFile(filename)
    if (Signal_DNN == "Inclusive"):
        ntrig_histo = file.Get('N_Triggers_pT%1.0f_%1.0f' %(pTbins[ipt],pTbins[ipt+1]))
    else:
        DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN))
        ntrig_histo = file.Get('N_DNN%i_Triggers_pT%1.0f_%1.0f' %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1]))
    NTriggers = 1
    if not(ntrig_histo == None):
        if (Signal_DNN):
            NTriggers = ntrig_histo.GetEntries()
        else:
            if (Use_Weights):
                NTriggers = ntrig_histo.GetBinContent(1)
            else:
                NTriggers = ntrig_histo.GetEntries()
    file.Close()
    return NTriggers

In [ ]:
def GetCorrelation_Npairs(filename,pt_min,pt_max, izt, Signal_DNN=True):
    
    file = ROOT.TFile(filename)
    
    if (Signal_DNN == "Inclusive"):
        histo2D = file.Get('Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                        %(pt_min[ipt],pt_max,100*zTbins[izt],100*zTbins[izt+1]))
    
    else:
        DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN)) #convert bool to DNN_1 (Sgn) or DNN_2 (Bkgd)
        
        histo2D = file.Get('DNN%i_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                            %(DNN_Rgn,pt_min,pt_max,100*zTbins[izt],100*zTbins[izt+1]))
        
    N_Pairs = histo2D.GetEntries()
    return N_Pairs

In [ ]:
#### N_Trig_Array_pPb = np.zeros(NpT)
N_Trig_Array_pp = np.zeros(NpT)

N_Trig_Array_Purity_pPb = np.zeros(NpT)
N_Trig_Array_Purity_pp = np.zeros(NpT)

N_Pairs_Array_pPb = np.zeros((NpT,NzT))
N_Pairs_Array_pp = np.zeros((NpT,NzT))

N_Pairs_Purity_pPb = np.zeros((NpT,NzT))
N_Pairs_Purity_pp = np.zeros((NpT,NzT))


for file,SYS in zip(Files,Systems):
    
    for ipt in range (NpT):
        
        vars()["N_Trig_Array_%s"%(SYS)][ipt] = Get_NTriggers(file, ipt, True)
        vars()["N_Trig_Array_Purity_%s"%(SYS)][ipt] = vars()["N_Trig_Array_%s"%(SYS)][ipt]*purity[ipt]
        
        for izt in range (NzT):
            vars()["N_Pairs_Array_%s"%(SYS)][ipt][izt] = GetCorrelation_Npairs(file,pTbins[ipt],pTbins[ipt+1], izt, True)
            vars()["N_Pairs_Purity_%s"%(SYS)][ipt][izt] = (vars()["N_Pairs_Array_%s"%(SYS)][ipt][izt]*purity[ipt])/vars()["N_Trig_Array_%s"%(SYS)][ipt]
        
print(N_Trig_Array_Purity_pp)
print(N_Trig_Array_pp)

In [ ]:
for SYS in Systems:
    if (SYS=="pPb"):
        sys_color="blue"
    else:
        sys_color="red"
    for ipt in range (NpT):
        plt.figure(figsize=(10,7)) 
        empt, = plt.plot([], [],' ')
        bar_zT_pPb = plt.bar(zT_centers,N_Pairs_Purity_pPb[ipt],zT_widths,color=sys_color,alpha=0.5)
        bar_zT_pp = plt.bar(zT_centers,N_Pairs_Purity_pp[ipt],zT_widths,color=sys_color)
        plt.legend([bar_zT_pPb,bar_zT_pp,empt],["pPb No. Pairs","pp No. Pairs",r'%1.0f < $p_\mathrm{T}^{\mathrm{trig}}$ < %1.0f GeV/$c$'%(pTbins[ipt],pTbins[ipt+1])])
        plt.title("No of Pairs in zT bins in (Purity Scaled)")

In [ ]:
for SYS in Systems:
    if (SYS=="pPb"):
        sys_color="blue"
    else:
        sys_color="red"
    for ipt in range (NpT):
        plt.figure(figsize=(10,7)) 
        empt, = plt.plot([], [],' ')
        bar_pT = plt.bar(pTbins[:NpT],vars()["N_Trig_Array_Purity_%s"%(SYS)],-pT_widths,align='edge',color=sys_color)
        plt.legend([bar_pT,empt],["No. Triggers (P)",r'%1.0f < $p_\mathrm{T}^{\mathrm{trig}}$ < %1.0f GeV/$c$'%(pTbins[ipt],pTbins[ipt+1])])
        plt.title("No of Triggers in pT bins (Purity Scaled) in %s"%(SYS))

In [ ]:
low_pp = []
high_pp = []
low_pPb = []
high_pPb = []
ztstring = []
for izt in range (zT_offset,NzT+zT_offset):
    ztb = izt-zT_offset
    ztstring.append("%1.2f-%1.2f"%(zTbins[izt],zTbins[izt+1]))
    low_pp.append(GetCorrelation_Npairs('InputData/17q_SE_NN_Correlation.root',12,15,izt,True))
    high_pp.append(GetCorrelation_Npairs('InputData/17q_SE_NN_Correlation_HighpT.root',15,20,izt,True))
    low_pPb.append(GetCorrelation_Npairs('InputData/13def_SE_NN_Correlation.root',12,15,izt,True))
    high_pPb.append(GetCorrelation_Npairs('InputData/13def_SE_NN_Correlation_HighpT.root',15,20,izt,True))

print(ztstring)
print(low_pp)
print(high_pp)
print(low_pPb)
print(high_pPb)

# Look at simple $\Delta \phi$ Projections

In [ ]:
# pPb_File = 'InputData/13d_MB_0GeV_NN_15_20.root'
# pp_File = 'InputData/17q_MB_0Gev_NN_15_20.root'

In [ ]:
#####Obtains the phi projection within the eta region (|eta|<0.6)#####
def GetPhiProj(filename,prfx,ipt, izt, Signal_DNN=True):
    
    file = ROOT.TFile(filename)

    eta_max = 1.2 #Assuming symmetric eta range
    
    if (Signal_DNN == "Inclusive"):
        histo2D = file.Get('Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                        %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],100*zTbins[izt+1]))
        
        Eta_Axis = histo2D.GetYaxis()
        PhiProjection = histo2D.ProjectionX('Inclusive_PhiProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                      %(pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],
                                        100*zTbins[izt+1]),Eta_Axis.FindBin(-eta_max),Eta_Axis.FindBin(eta_max))
    
    else:
        DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN)) #convert bool to DNN_1 (Sgn) or DNN_2 (Bkgd)
        histo2D = file.Get('DNN%i_Correlation__pT%1.0f_%1.0f__zT%1.0f_zT%1.0f' 
                            %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],100*zTbins[izt+1]))
        
        Eta_Axis = histo2D.GetYaxis()
        PhiProjection = histo2D.ProjectionX('DNN%i_PhiProjection__pT_%1.0f_%1.0f__zt_%1.0f_%1.0f' 
                                      %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1],100*zTbins[izt],
                                        100*zTbins[izt+1]),Eta_Axis.FindBin(-eta_max),Eta_Axis.FindBin(eta_max))
                                            
    PhiProjection.SetDirectory(0)
    PhiProjection.Rebin(2)
    PhiProjection.Scale(1.0/(2*eta_max))
    
    #per trigger yield
#     ntriggers = Get_NTriggers(filename,ipt, Signal_DNN)
#     if not(ntriggers == None):
#         PhiProjection.Scale(1.0/ntriggers)
    
    file.Close()
    
    Phi_Array = np.zeros(len(delta_phi_centers))
    Phi_Error_Array = np.zeros(len(delta_phi_centers))
    for bin in range(2,9):
        Phi_Array[bin-2] = PhiProjection.GetBinContent(bin)
        Phi_Error_Array[bin-2] = PhiProjection.GetBinError(bin)
    
    return Phi_Array, Phi_Error_Array

In [ ]:
Systems = ["pp","p-Pb"]
Files = [pp_File,pPb_File]

for sys,ifile in zip(Systems,Files):
    fig = plt.figure(figsize=(34,17))
    for ipt in range (3):
        if (ipt > 0): continue
        ipt = ipt+2
        for izt in range (zT_offset,NzT+zT_offset):
            
            ztb = izt-zT_offset

            Sig_Phi_Array, Sig_Phi_Error_Array = GetPhiProj(ifile,sys,ipt,izt,True)
            Bkg_Phi_Array, Bkg_Phi_Error_Array = GetPhiProj(ifile,sys,ipt,izt,False)
                                            
                                        #--------------plot--------------------#
           
            if (NzT ==4):
                ax = fig.add_subplot(2,4,(2*ztb+1))
            elif (NzT ==6):
                ax = fig.add_subplot(3,4,(2*ztb+1))
                
            fsize = 20

            #sig
            ax.plot(delta_phi_centers,Sig_Phi_Array,'bo',ms=10)
            s_plot = ax.errorbar(delta_phi_centers,Sig_Phi_Array,xerr=phi_width,yerr=Sig_Phi_Error_Array,
                                 fmt=None,ecolor='b',label='Signal Region (stat. error)')

            plt.xlabel(r'|$\Delta \varphi$|',fontsize=fsize+4)
            plt.xticks(fontsize=(fsize))
            plt.xlim((0.39269908169872414,3.14159))
            plt.ylabel(r'$1/N_{\mathrm{trig}} \: \: \mathrm{d}N/\mathrm{d}\Delta\eta$',fontsize=fsize+2)
            plt.ylim((0,1.2*max(Sig_Phi_Array)))
            empt, = ax.plot([], [], ' ')
            empt2, = ax.plot([],[],' ')
            plt.yticks(fontsize=fsize-5)

            fill_x = [0,3.14159]
#             s_z_line = ax.fill_between(fill_x, Sig_Z_Value-Sig_Z_Error,Sig_Z_Value+Sig_Z_Error,interpolate=False,edgecolor='cyan',linewidth=0.0, alpha=0.3,facecolor='cyan')
#             s_le_line = ax.fill_between(fill_x, S_LE-S_LE_Error,S_LE+S_LE_Error,interpolate=False,edgecolor='grey',linewidth=0.0, alpha=0.5,facecolor='grey')

            leg = ax.legend([s_plot,empt,empt2],['Signal Region (stat. error)','0.8 <|$\Delta\eta$| < 1.4',
                'UE Estimate',r'%1.2f < $z_\mathrm{T}$ < %1.2f'%(zTbins[izt],zTbins[izt+1]),
                r'15 < $p_\mathrm{T}^{\mathrm{trig}}$ < 20 GeV/$c$'],loc='best',
                title = "Alice Preliminary 5 TeV %s"%(sys),fontsize=14,frameon=False,numpoints=1)
            plt.setp(leg.get_title(),fontsize=20)
#             if (sys == 'pp'):
#                 leg.set_title("ALICE Preliminary, $\sqrt{s}=$5 TeV %s"%(sys))
#             else:
#                 leg.set_title("ALICE Preliminary, $\sqrt{s_{\mathrm{_{NN}}}}=$5 TeV %s"%(sys))                
#             plt.setp(leg.get_title(),fontsize=14)


            #bkg
            if (NzT ==4):
                ax = fig.add_subplot(2,4,(2*ztb+2))
            elif (NzT ==6):
                ax = fig.add_subplot(3,4,(2*ztb+2))
                
            #ax = fig.add_subplot(1,2,1)
            plt.xlabel(r'|$\Delta \varphi$|',fontsize=fsize+4)
            plt.xticks(fontsize=(fsize))
            plt.xlim((0.39269908169872414,3.14159))
            plt.ylabel(r'$1/N_{\mathrm{trig}} \: \: \mathrm{d}N/\mathrm{d}\Delta\eta$',fontsize=fsize+2)
            plt.ylim((0,1.2*max(Bkg_Phi_Array)))
            plt.yticks(fontsize=fsize-5)

            fill_x = [0,3.14149]
#             b_z_line = plt.fill_between(fill_x, Bkg_Z_Value-Bkg_Z_Error,Bkg_Z_Value+Bkg_Z_Error,interpolate=False,edgecolor='cyan',linewidth=0.0, alpha=0.3,facecolor='cyan')
#             b_le_line = plt.fill_between(fill_x, Bkg_LE-Bkg_LE_Error,Bkg_LE+Bkg_LE_Error,interpolate=False,edgecolor='grey',linewidth=0.0, alpha=0.5,facecolor='grey')

            ax.plot(delta_phi_centers,Bkg_Phi_Array,'ro',ms=10)
            b_plot = ax.errorbar(delta_phi_centers,Bkg_Phi_Array,xerr=phi_width,yerr=Bkg_Phi_Error_Array,fmt=None,ecolor='r')

            leg = ax.legend([b_plot,empt,empt2],['Bkg Region (stat. error)',r'%1.2f < $z_\mathrm{T}$ < %1.2f'%(zTbins[izt],
                zTbins[izt+1]),r'15 < $p_\mathrm{T}^{\mathrm{trig}}$ < 20 GeV/$c$'],loc='best',
                title = "Alice Preliminary 5 TeV %s"%(sys),fontsize=14,frameon=False,numpoints=1)
            plt.setp(leg.get_title(),fontsize=20)

     
    print("")
    #return



In [ ]:
print(pTbins[2])

In [ ]:
#Shower_Shape = "NN"
Shower_Shape = "L0"

if (Shower_Shape=="NN"):
    File = 'InputData/13d_SE_NN_Correlation.root'
elif (Shower_Shape=="L0"):
    File = 'InputData/13d_SE_L0_Correlation.root'
file = ROOT.TFile(File)

N_Sig_histo = file.Get("N_DNN1_Triggers_pT12_15")
N_Sig = N_Sig_histo.GetEntries()
N_BKG_histo = file.Get("N_DNN2_Triggers_pT12_15")
N_BKG = N_BKG_histo.GetEntries()


SIG = file.Get("Signal_pT_Dist")
SIG.Rebin(5)
print(SIG.GetEntries())
BKGD = file.Get("BKGD_pT_Dist")
BKGD.Rebin(5)
print(BKGD.GetEntries())
BKGD_W = file.Get("BKGD_pT_Dist_Weighted")
BKGD_W.Rebin(5)

#SIG.Rebin(10)
Signal = root_numpy.hist2array(SIG)
S_Scale = Signal.sum(axis=0)
Signal = Signal/S_Scale


#BKGD.Rebin(10)
Background = root_numpy.hist2array(BKGD)
B_Scale = Background.sum(axis=0)
Background = Background/B_Scale

#BKGD_W.Rebin(10)
Background_W = root_numpy.hist2array(BKGD_W)
B_Scale_W = Background_W.sum(axis=0)
Background_W = Background_W/B_Scale_W

N_Bins = SIG.GetSize() - 2 #Ignero under/overflow
S_Error = np.zeros(N_Bins)
B_Error = np.zeros(N_Bins)
BW_Error = np.zeros(N_Bins)
for i in range(1,N_Bins+1):
    S_Error[i-1] = SIG.GetBinError(i)
    print(SIG.GetBinError(i))
    B_Error[i-1] = BKGD.GetBinError(i)
    BW_Error[i-1] = BKGD_W.GetBinError(i)
    
S_Error = S_Error/S_Scale
B_Error = B_Error/B_Scale
BW_Error = BW_Error/B_Scale_W
    
print(Signal.size)
print(S_Error.size)

pT_Axis = np.linspace(10,40,N_Bins)

fig = plt.figure(figsize=(10,7))
if (Shower_Shape=="NN"):
    plt.errorbar(pT_Axis,Signal,yerr = S_Error,label="Signal Region Clusters (0.55 < NN < 0.85)")
    plt.errorbar(pT_Axis,Background,yerr = B_Error,label="Background Region Clusters (NN < 0.4)")
    plt.errorbar(pT_Axis,Background_W,yerr = BW_Error,fmt = "--r",label="Weighted Background Clusters (NN < 0.4)")
    plt.title(r"Isolated Cluster $p_{\mathrm{T}}$ Distribution  $DNN$")

elif (Shower_Shape=="L0"):
    plt.errorbar(pT_Axis,Signal,yerr = S_Error,label="Signal Region Clusters ($\lambda^{2}_{0}$ < 0.3)")
    plt.errorbar(pT_Axis,Background,yerr = B_Error,label="Background Region Clusters ($\lambda^{2}_{0}$ > 0.4)")
    plt.errorbar(pT_Axis,Background_W,yerr = BW_Error,fmt = "--r",label="Weighted Bkg Region Clusters ($\lambda^{2}_{0}$ > 0.4)")
    plt.title(r"Isolated Cluster $p_{\mathrm{T}}$ Distribution  $\lambda^{2}_{0}$")
    #plt.ylim(0,0.01)
    
plt.xlabel(r'Cluster $p_{\mathrm{T}}$')
plt.ylabel(r'$\frac{1}{N_{\gamma}}\cdot dN$')
plt.legend()
fig.savefig('pics/Cluster_pT_Weighted.pdf', bbox_inches='tight')



In [ ]:
#Shower_Shape = "NN"
Shower_Shape = "L0"

if (Shower_Shape=="NN"):
    File = 'InputData/17q_SE_NN_Correlation.root'
elif (Shower_Shape=="L0"):
    File = 'InputData/13d_SE_L0_Correlation.root'
file = ROOT.TFile(File)

SIG = file.Get("hweight")
print(SIG)
print(SIG.GetEntries())
Signal = root_numpy.hist2array(SIG)
S_Error = np.zeros(40)
for i in range(1,41):
    S_Error[i-1] = SIG.GetBinError(i)
    

pT_Axis = np.linspace(10,50,40)
fig = plt.figure(figsize=(10,7))

if (Shower_Shape=="NN"):
    plt.errorbar(pT_Axis,Signal,yerr = S_Error,label="Signal Clusters ($0.5 < NN < 0.85)")
    
elif (Shower_Shape=="L0"):
    plt.errorbar(pT_Axis,Signal,yerr = S_Error,label="Cluster pT weights")
    plt.title(r"Isolated Cluster $p_{\mathrm{T}}$ Weights")

    
plt.xlabel(r'Cluster $p_{\mathrm{T}}$')
plt.ylabel(r'Cluster $p_{\mathrm{T}}$ Weight')
plt.xlim(12,40)
#plt.ylim(0,1.2)
plt.legend()
fig.savefig('pics/Weights.pdf', bbox_inches='tight')




### Compare ME Inclusive and Signal Correlations

In [ ]:
Shower_Shape = "NN"

if (Shower_Shape=="NN"):
    File = 'InputData/13f_MB_0GeV_Skim_Correlation_NN.root'
elif (Shower_Shape=="L0"):
    File = 'InputData/17q_MB_0GeV_Skim_Correlation.root'
file = ROOT.TFile(File)

SIG_2D = file.Get("DNN1_Correlation__pT12_15__zT12_zT18")
BKG_2D = file.Get("DNN2_Correlation__pT12_15__zT12_zT18")
INC_2D = file.Get("Correlation__pT12_15__zT12_zT18")

SIG = SIG_2D.ProjectionX("Sig_Phi",0,-1)
BKG = BKG_2D.ProjectionX("Bkg_Phi",0,-1)
INC = INC_2D.ProjectionX("Inc_Phi",0,-1)

SIG.Divide(INC)
BKG.Divide(INC)

Signal = root_numpy.hist2array(SIG)
Background = root_numpy.hist2array(BKG)

N_Bins = SIG.GetSize() - 2 #Ignero under/overflow
print(N_Bins)
S_Error = np.zeros(N_Bins)
B_Error = np.zeros(N_Bins)
for i in range(1,N_Bins+1):
    S_Error[i-1] = SIG.GetBinError(i)
    B_Error[i-1] = BKG.GetBinError(i)

Corr_Axis = np.linspace(0,3.14159,N_Bins)    

print(SIG)

if (Shower_Shape=="NN"):
    plt.errorbar(Corr_Axis,Signal,yerr = S_Error,label="Signal Clusters ($0.5 < NN < 0.85)")
    plt.title(r"Isolated Cluster $p_{\mathrm{T}}$ Distribution  $DNN$")
    #plt.ylim(0.5,0.58)
    plt.ylim(0.0,0.33)

elif (Shower_Shape=="L0"):
    plt.errorbar(Corr_Axis,Signal,yerr = S_Error,label="Signal Clusters ($\lambda^{2}_{0}$ < 0.4)")

Signal = root_numpy.hist2array(SIG)
Background = root_numpy.hist2array(SIG)





In [ ]:
File_NN = 'InputData/17q_MB_0GeV_Skim_Correlation.root'
#File_NN = 'InputData/13f_SE_NN_Correlation_GMB_Ratio.root'
File_L0 = 'InputData/17q_MB_0GeV_Skim_Correlation_L0.root'

file_NN = ROOT.TFile(File_NN)
file_L0 = ROOT.TFile(File_L0)


SIG_2D_NN = file_NN.Get("DNN1_Correlation__pT12_15__zT12_zT18")
SIG_2D_L0 = file_L0.Get("DNN1_Correlation__pT12_15__zT12_zT18")

BKG_2D_NN = file_NN.Get("DNN2_Correlation__pT12_15__zT12_zT18")
BKG_2D_L0 = file_L0.Get("DNN2_Correlation__pT12_15__zT12_zT18")

c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)
BKG_2D_NN.Draw("SURF2")
c.Draw()

INC_2D_NN = file_NN.Get("Correlation__pT12_15__zT12_zT18")
INC_2D_L0 = file_L0.Get("Correlation__pT12_15__zT12_zT18")

SIG_NN = SIG_2D_NN.ProjectionX("Sig_Phi",0,-1)
SIG_L0 = SIG_2D_L0.ProjectionX("Sig_Phi",0,-1)

BKG_NN = BKG_2D_NN.ProjectionX("Bkg_Phi",0,-1)
BKG_L0 = BKG_2D_L0.ProjectionX("Bkg_Phi",0,-1)

INC_NN = INC_2D_NN.ProjectionX("Inc_Phi",0,-1)
INC_L0 = INC_2D_L0.ProjectionX("Inc_Phi",0,-1)

SIG_L0.Divide(INC_NN)
BKG_NN.Divide(BKG_L0)

Signal = root_numpy.hist2array(SIG_L0)
print(Signal)
Background = root_numpy.hist2array(BKG_NN)

N_Bins = SIG_NN.GetSize() - 2 #Ignero under/overflow
print(N_Bins)
S_Error = np.zeros(N_Bins)
B_Error = np.zeros(N_Bins)
for i in range(1,N_Bins+1):
    S_Error[i-1] = SIG_NN.GetBinError(i)
    B_Error[i-1] = BKG_NN.GetBinError(i)

Corr_Axis = np.linspace(0,3.14159,N_Bins)    

#plt.errorbar(Corr_Axis,Signal,yerr = S_Error,label="Signal Clusters ($0.5 < NN < 0.85)")
#plt.title(r"Isolated Cluster $p_{\mathrm{T}}$ Distribution  $DNN$")

plt.errorbar(Corr_Axis,Signal,yerr = S_Error,label="Signal Clusters ($0.5 < NN < 0.85)")
plt.title(r"Isolated Cluster $p_{\mathrm{T}}$ Distribution  $DNN$")
#plt.ylim(0.99,1.01)
plt.ylim(0.2,0.4)
    
print("hi")

In [ ]:
def Get_NTriggers(filename,ipt, Signal_DNN=True): 
    file = ROOT.TFile(filename)
    if (Signal_DNN == "Inclusive"):
        ntrig_histo = file.Get('N_Triggers_pT%1.0f_%1.0f' %(pTbins[ipt],pTbins[ipt+1]))
    else:
        DNN_Rgn = int(Signal_DNN) + 2*(1-int(Signal_DNN))
        ntrig_histo = file.Get('N_DNN%i_Triggers_pT%1.0f_%1.0f' %(DNN_Rgn,pTbins[ipt],pTbins[ipt+1]))
    NTriggers = 1
    if not(ntrig_histo == None):
        if (Signal_DNN):
            NTriggers = ntrig_histo.GetEntries()
        else:
            if (Use_Weights):
                NTriggers = ntrig_histo.GetBinContent(1)
            else:
                NTriggers = ntrig_histo.GetEntries()
    file.Close()
    return NTriggers

In [ ]:
if (Shower_Shape=="NN"):
    File = 'InputData/13f_MB_0GeV_Skim_Correlation_NN.root'
elif (Shower_Shape=="L0"):
    File = 'InputData/17q_MB_0GeV_Skim_Correlation.root'
file = ROOT.TFile(File)

pT_Bins = [12,15,19,26,40]
N_pT_Bins = len(pT_Bins)
pT_Centres =  np.zeros[N_pT_Bins]
N_Sig_Triggers = np.zeros(N_pT_Bins)
N_Bkg_Trigs = np.zeros(N_pT_Bins)
N_Pairs = np.zeros((N_pT_Bins,NzT))

for ipt in range (N_pT_Bins):
    pT_Centers[ipt] = (pT_Bins[ipt] + pT_Bins[ipt+1])/2
    N_Sig_Trigs[ipt] = Get_NTriggers(File,ipt,True)
    N_Bkg_Trigs[ipt] = Get_NTriggers(File,ipt,True)
    
plt.plot(pT_Centers, N_Sig_Trigs)